## 8.2. test simulations

This notebook applies our pseudotime method on a simulation dataset. Note: the R code in script 3.1. has been converted to Python script for this purpose.

In [ ]:
### load libraries

# standard libraries
import numpy as np

Function `response_pseudotime` is the same as the original function in script 3.1. 

In [ ]:
### create function of regression model without intercept
def response_pseudotime(X,Tvect):
    """
    Calculate pseudotime using a regression model without an intercept.

    Parameters:
    - X: numpy array, count matrix (cells x genes)
    - Tvect: numpy array, vector of experimental timepoints for each cell

    Returns:
    - PT: numpy array, pseudotime values
    """
    
    # get covariance matrix
    XXt = np.dot(X, X.T)

    # find the inverse of XXt
    XXtinv = np.linalg.inv(XXt)

    # find the transformation (W) in gene space
    W = np.dot(np.dot(XXtinv, X), Tvect)

    # derive pseudotime (PT) from transformation
    PT = np.dot(W.T, X)
    
    return(PT)

Here, we calculate pseudotime for different types of simulations to see how well we are able to recover the pseudotemporal order and the temporal dynamics of individual genes.

Here, we compare our original regression model with a model with intercept included. 

In [ ]:
### create function of regression model with intercept
def calculate_pseudotime_intercept(X, Tvect):
    """
    Calculate pseudotime using a regression model with an intercept.

    Parameters:
    - X: numpy array, count matrix (cells x genes)
    - Tvect: numpy array, vector of experimental timepoints for each cell

    Returns:
    - PT: numpy array, pseudotime values
    """

    # add a column of ones for the intercept
    X_with_intercept = np.column_stack((np.ones(X.shape[0]), X))

    # get covariance matrix
    XXt = np.dot(X_with_intercept, X_with_intercept.T)

    # find the inverse of XXt
    XXtinv = np.linalg.inv(XXt)

    # find the transformation (W) in genes space
    W = np.dot(np.dot(XXtinv, X_with_intercept), Tvect)

    # derive pseudotime (PT) from transformation
    PT = np.dot(W.T, X_with_intercept)

    return PT